In [16]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド
    pad : パディング
    Returns
    -------
    col : 2次元配列
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [17]:
import sys, os
sys.path.append(os.pardir)
#from common.util import im2col
import numpy as np

x1 = np.random.rand(1, 3, 7, 7)  # batch_size, channel_size, height, width
coll = im2col(x1, 5, 5, stride=1, pad=0)
print(coll.shape)

x2 = np.random.rand(10, 3, 7, 7)  # batch_size, channel_size, height, width
coll = im2col(x2, 5, 5, stride=1, pad=0)
print(coll.shape)


(9, 75)
(90, 75)


In [5]:
class Convolution:
    
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T  
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)  #?
        
        return out

In [7]:
class Pooling:

    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (H - self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad) #?
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2) 
        
        return out 

In [20]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class SimpleConvNet:
    """単純なConvNet
    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 入力サイズ（MNISTの場合は784）
    hidden_size_list : 隠れ層のニューロンの数のリスト（e.g. [100, 100, 100]）
    output_size : 出力サイズ（MNISTの場合は10）
    activation : 'relu' or 'sigmoid'
    weight_init_std : 重みの標準偏差を指定（e.g. 0.01）
        'relu'または'he'を指定した場合は「Heの初期値」を設定
        'sigmoid'または'xavier'を指定した場合は「Xavierの初期値」を設定
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """損失関数を求める
        引数のxは入力データ、tは教師ラベル
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """勾配を求める（数値微分）
        Parameters
        ----------
        x : 入力データ
        t : 教師ラベル
        Returns
        -------
        各層の勾配を持ったディクショナリ変数
            grads['W1']、grads['W2']、...は各層の重み
            grads['b1']、grads['b2']、...は各層のバイアス
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """勾配を求める（誤差逆伝搬法）
        Parameters
        ----------
        x : 入力データ
        t : 教師ラベル
        Returns
        -------
        各層の勾配を持ったディクショナリ変数
            grads['W1']、grads['W2']、...は各層の重み
            grads['b1']、grads['b2']、...は各層のバイアス
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]            self.layers[key].b = self.params['b' + str(i+1)]

SyntaxError: invalid syntax (<ipython-input-20-95935e9e0ab8>, line 150)

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
#from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 処理に時間のかかる場合はデータを削減 
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# パラメータの保存
network.save_params("params.pkl")
print("Saved Network Parameters!")

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.30022606535
=== epoch:1, train acc:0.233, test acc:0.223 ===
train loss:2.29939960027
train loss:2.29518729673
train loss:2.28901755095
train loss:2.28401148956
train loss:2.2736991712
train loss:2.25535804877
train loss:2.24867227311
train loss:2.22967144857
train loss:2.19619425257
train loss:2.20343527963
train loss:2.13860469709
train loss:2.10578110039
train loss:2.05440647813
train loss:2.01609557897
train loss:1.96961235878
train loss:1.89920095091
train loss:1.82397684957
train loss:1.77226185804
train loss:1.71560318626
train loss:1.56903123155
train loss:1.56055646908
train loss:1.4178563478
train loss:1.41959241141
train loss:1.25414716698
train loss:1.31435785899
train loss:1.17781131257
train loss:1.06830175049
train loss:1.02586807376
train loss:0.925791556847
train loss:0.796584524533
train loss:0.651982516349
train loss:0.800595639673
train loss:0.660801734357
train loss:0.723120867688
train loss:0.71272365228
train loss:0.527912729159
train loss:0.75631101

train loss:0.191296929569
train loss:0.295282104981
train loss:0.209872904586
train loss:0.191763128053
train loss:0.15700236896
train loss:0.176147450559
train loss:0.330009939541
train loss:0.2103688827
train loss:0.161217846957
train loss:0.229261259993
train loss:0.182834336127
train loss:0.209428137205
train loss:0.214959345191
train loss:0.191248715112
train loss:0.177145898637
train loss:0.199832671284
train loss:0.130380999931
train loss:0.147767705275
train loss:0.183753234632
train loss:0.285270625068
train loss:0.218649323463
train loss:0.0956122653784
train loss:0.329575538034
train loss:0.258642068043
train loss:0.088095762121
train loss:0.155145220734
train loss:0.258811649672
train loss:0.142258777975
train loss:0.170925292729
train loss:0.148148422602
train loss:0.182845814089
train loss:0.23996304932
train loss:0.26395463165
train loss:0.0790001956186
train loss:0.0843713538818
train loss:0.113387571144
train loss:0.142691209818
train loss:0.088336505748
train loss:0.1

train loss:0.0642272261165
train loss:0.0658663492071
train loss:0.0835582863268
train loss:0.0561899897386
train loss:0.0776791892761
train loss:0.0938779982774
train loss:0.136202647334
train loss:0.0608956774945
train loss:0.111180521486
train loss:0.135631608664
train loss:0.0433381814821
train loss:0.0378916241159
train loss:0.177992420223
train loss:0.13115419262
train loss:0.0331836583585
train loss:0.068482632552
train loss:0.184078603763
train loss:0.195856731846
train loss:0.109064783832
train loss:0.151460146425
train loss:0.0771008240149
train loss:0.0432840427513
train loss:0.0895077823285
train loss:0.119295973543
train loss:0.0554319679783
train loss:0.107696820335
train loss:0.090546560789
train loss:0.0714192112339
train loss:0.134975966321
train loss:0.0371736702494
train loss:0.0993704240586
train loss:0.105112354408
train loss:0.0684604349434
train loss:0.0603047032183
train loss:0.0957707388666
train loss:0.0727634249781
train loss:0.0818024088964
train loss:0.0705

train loss:0.0548549446636
train loss:0.0410716221878
train loss:0.0734490585743
train loss:0.108126165646
train loss:0.054153697586
train loss:0.109167901003
train loss:0.102067386447
train loss:0.0653992432738
train loss:0.0529272260444
train loss:0.146021857716
train loss:0.0584628906296
train loss:0.137915014557
train loss:0.0563688156788
train loss:0.0268585703307
train loss:0.0382960806962
train loss:0.0398152599098
train loss:0.0572790534878
train loss:0.0941394328598
train loss:0.0264503602162
train loss:0.0888701167235
train loss:0.0671687544133
train loss:0.0962112121665
train loss:0.0946643212992
train loss:0.0315960793191
train loss:0.0696253142673
train loss:0.0520464538591
train loss:0.0553067145907
train loss:0.0471375680341
train loss:0.024366084197
train loss:0.143714428123
train loss:0.0474405299991
train loss:0.0430672742677
train loss:0.0276810077962
train loss:0.0560115813726
train loss:0.12808406309
train loss:0.10335570885
train loss:0.0776982728523
train loss:0.